In [244]:
import pickle
from transformers import BertTokenizer
import re
import pandas as pd
from string import punctuation
from math import log

import torch
import 

OSError: [WinError 127] The specified procedure could not be found

In [4]:
BERT_preprocessing_tokenizer = "bert-base-uncased"

In [240]:
# Read in raw data
with open('data/docs_demo_processed_10.pickle', 'rb') as f:
    docs = pickle.load(f)

In [6]:
# Process as sentence

def preprocess_text(sentence):
    sentence = sentence.lower()  # Lowercase text
    sentence = re.sub(f"[{re.escape(punctuation)}]", "", sentence)  # Remove punctuation
    sentence = " ".join(sentence.split())  # Remove extra spaces, tabs, and new lines
    return(sentence)

In [7]:
# Tokenize as sentence
tokenizer = BertTokenizer.from_pretrained(BERT_preprocessing_tokenizer)

def tokenize_text(sentence):
    sentence = tokenizer.tokenize(sentence)
    return(sentence)

In [10]:
# sentence = ['the', 'truth', 'of', 'the', 'matter', 'is', 'that', 'the', 'titanic', 'even', 'though', 'its', 'breaking', 'all', 'sorts', 'of', 'box', 'office', 'records', 'its', 'not', 'the', 'most', 'exciting', 'story', 'from', 'the', 'sea']

# Add sentence to vocab (with frequency)

def add_sentence_to_vocab(sentence, vocab):
    for word in sentence:
        if word in vocab.keys():
            vocab[word] += 1
        else:
            vocab[word] = 1

In [126]:
# List of tokenized words for one document

# Loop through sentences, do preprocessing & tokenization then add tokens to doc list and doc vocab
def preprocess_doc(doc):
    tokens_from_doc = list()
    doc_vocab = {}

    for sentence in doc:
        sentence = preprocess_text(sentence)
        sentence = tokenize_text(sentence)
        add_sentence_to_vocab(sentence, doc_vocab)
        [tokens_from_doc.append(token) for token in sentence]
    
    return tokens_from_doc, doc_vocab

In [135]:
def add_new_tokens_to_corpus_adj_list(doc_vocab, corpus_adj_list=corpus_adj_list, printout = False):
    new_tokens = set(doc_vocab.keys()).difference(set(corpus_adj_list.keys()))

    print('Doc has %d new tokens' % len(new_tokens))
    for token in new_tokens:
        corpus_adj_list[token] = {}

In [133]:
def get_doc_windows(tokens_from_doc, window_size=window_size, padding=padding):
    doc_windows = []
    for i in range(0, len(tokens_from_doc), window_size - padding):
        doc_windows.append(tokens_from_doc[i:i + window_size])
    return(doc_windows)

def token_adj_from_window(window):

    for i in range(0, len(window) - 1):

        for j in range(i + 1, len(window)):
            
            if window[j] in corpus_adj_list[window[i]].keys():
                corpus_adj_list[window[i]][window[j]] += 1

            else:
                corpus_adj_list[window[i]][window[j]] = 1


In [193]:
def add_doc_vocab_to_corpus(doc_vocab, corpus_vocab=corpus_vocab):

    for token in doc_vocab.keys():
        if token in corpus_vocab.keys():
            corpus_vocab[token] += doc_vocab[token]

        else: 
            corpus_vocab[token] = 1

## Execution

In [242]:
corpus_vocab = {}
corpus_adj_list = {}
corpus_windows_count = 0
corpus_windows = []
corpus_token_appeared_in_window = {}
corpus_token_pair_appeared_in_window = {}

window_size = 20
padding = 5

In [243]:
for doc in docs:

    tokens_from_doc, doc_vocab = preprocess_doc(doc)

    add_doc_vocab_to_corpus(doc_vocab, corpus_vocab=corpus_vocab)
    
    add_new_tokens_to_corpus_adj_list(doc_vocab, corpus_adj_list=corpus_adj_list, printout=True)

    doc_windows = get_doc_windows(tokens_from_doc, window_size=20, padding=5)

    for window in doc_windows:
        token_adj_from_window(window)
    
    for window in doc_windows: corpus_windows.append(window) 
    
    print('Corpus windows count = %d' % len(corpus_windows))
    print('Doc windows count = %d' % len(doc_windows))

Doc has 779 new tokens
Corpus windows count = 198
Doc windows count = 198
Doc has 590 new tokens
Corpus windows count = 408
Doc windows count = 210
Doc has 470 new tokens
Corpus windows count = 611
Doc windows count = 203
Doc has 158 new tokens
Corpus windows count = 672
Doc windows count = 61
Doc has 96 new tokens
Corpus windows count = 721
Doc windows count = 49
Doc has 503 new tokens
Corpus windows count = 851
Doc windows count = 130
Doc has 94 new tokens
Corpus windows count = 896
Doc windows count = 45
Doc has 103 new tokens
Corpus windows count = 952
Doc windows count = 56
Doc has 228 new tokens
Corpus windows count = 1162
Doc windows count = 210


In [ ]:
# This stores how many times a token appears in the pre-determined text windows (for MNPI calculation - 'correlation' between two tokens)
token_appeared_in_corpus_windows = {}

In [202]:
def token_count_in_windows(token, windows):
    return sum([token in window for window in windows])

def token_pair_count_in_windows(token_1, token_2, windows):
    return sum([(token_1 in window) & (token_2 in window) for window in windows])

In [223]:
token_count_in_windows('and', corpus_windows)

6881

In [239]:
mpni('and', 'then', corpus_windows)

0.12104326571261242

In [204]:
def probability_of_token_in_windows(token, windows):
    return round(sum([token in window for window in windows])/len(windows), 4)

def probability_of_token_pair_in_windows(token_1, token_2, windows):
    return round(sum([(token_1 in window) & (token_2 in window) for window in windows])/len(windows), 4)

In [215]:
token_1 = 'david'
token_2 = 'i'

def mpni(token_1, token_2, windows, printout = False):
    prob_both_tokens = probability_of_token_pair_in_windows(token_1, token_2, windows)

    if prob_both_tokens == 0:
        npmi = -1
        return(npmi)

    prob_token_1 = probability_of_token_in_windows(token_1, windows)
    prob_token_2 = probability_of_token_in_windows(token_2, windows)

    npmi = (-1/log(prob_both_tokens, 10))*log((prob_both_tokens/(prob_token_1*prob_token_2)), 10)

    if printout == True:
        print('#i = %d' % token_count_in_windows(token_1, windows))
        print('#j = %d' % token_count_in_windows(token_2, windows))
        print('#i,j = %d' % token_pair_count_in_windows(token_1, token_2, windows))
        print('p(i,j) = %.4f' % prob_both_tokens)
        print('log(p(i,j) = %.4f' % log(prob_both_tokens, 10))
        print('p(i) = %.4f' % prob_token_1)
        print('p(j) = %.4f' % prob_token_2)

    return(npmi)

mpni(token_1, token_2, corpus_windows, printout = True)

-1

In [99]:
adj = {'a':{'b':1, 'c':2}, 'e':{'c': 1}}

# adj['a']['d'] = 1

new = ['d', 'c', 'e']

In [95]:
for i in new:
    if i not in adj.keys():
        adj[i] = {}



In [97]:
print(adj)

{'a': {'b': 1, 'c': 2}, 'd': {}, 'c': {}, 'e': {}}


In [92]:
# vocab.keys()
print(adj_list)

{'the': {}, 'truth': {}, 'of': {}, 'matter': {}, 'is': {}, 'that': {}, 'titanic': {}, 'even': {}, 'though': {}, 'its': {}, 'breaking': {}, 'all': {}, 'sorts': {}, 'box': {}, 'office': {}, 'records': {}, 'not': {}, 'most': {}, 'exciting': {}, 'story': {}, 'from': {}, 'sea': {}}


In [35]:
vocab = set()

for sentence in docs[0]:
    sentence = preprocess_text(sentence)
    sentence = tokenize_text(sentence)
    sentence_vocab = set(sentence)
    vocab = vocab.union(sentence_vocab)

['david', 'gallo', 'this', 'is', 'bill', 'lange', 'im', 'dave', 'gallo']
['and', 'were', 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video']
['we', '##ve', 'got', 'some', 'of', 'the', 'most', 'incredible', 'video', 'of', 'titanic', 'that', '##s', 'ever', 'been', 'seen', 'and', 'were', 'not', 'going', 'to', 'show', 'you', 'any', 'of', 'it']
['the', 'truth', 'of', 'the', 'matter', 'is', 'that', 'the', 'titanic', 'even', 'though', 'its', 'breaking', 'all', 'sorts', 'of', 'box', 'office', 'records', 'its', 'not', 'the', 'most', 'exciting', 'story', 'from', 'the', 'sea']
['and', 'the', 'problem', 'i', 'think', 'is', 'that', 'we', 'take', 'the', 'ocean', 'for', 'granted']
['when', 'you', 'think', 'about', 'it', 'the', 'oceans', 'are', '75', 'percent', 'of', 'the', 'planet']
['most', 'of', 'the', 'planet', 'is', 'ocean', 'water']
['the', 'average', 'depth', 'is', 'about', 'two', 'miles']
['part', 'of', 'the', 'problem', 'i', 'think', 'is', 'we', 'stan

In [39]:
len(vocab)
print(vocab)

{'##llie', '700', '##fide', 'our', 'fly', 'sun', 'comes', 'against', 'waiting', 'cameras', 'death', 'hell', 'imagery', 'though', 'lives', 'problems', 'begin', 'catch', 'proposing', '##e', 'oct', 'going', 'new', 'case', 'fingernails', 'eventually', 'dave', 'oh', 'fragile', 'end', 'face', 'beautiful', 'getting', 'watch', 'year', 'an', 'tube', 'lies', 'front', 'crab', 'started', 'from', '##cence', 'into', 'clinging', 'springs', 'make', 'down', '##mos', 'tentacles', 'average', 'do', 'look', 'percent', 'has', 'hands', 'ones', 'closer', 'listen', 'tomorrow', 'worm', 'as', 'hand', 'get', 'rich', 'head', 'thrive', 'hara', 'explain', '##ck', 'inches', 'surface', 'years', 'dot', 'contact', 'ears', 'real', 'tiny', 'd', 'with', 'note', 'david', 'hear', '##ivate', '70s', 'when', 'glaciers', 'understood', 'in', 'simple', 'those', 'big', 'tell', 'hydro', '##et', 'and', 'ribbon', 'developed', 'providing', 'support', 'true', 'familiar', 'system', 'good', 'story', 'last', 'range', 'his', 'rainforest', '

In [ ]:
from textblob import TextBlob

tokens = TextBlob(raw_data).words

In [ ]:
print(tokens)

['david', 'gallo', 'this', 'is', 'bill', 'lange', 'i', "'m", 'dave', 'gallo', 'and', 'we', "'re", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video', 'we', "'ve", 'got', 'some', 'of', 'the', 'most', 'incredible', 'video', 'of', 'titanic', 'that', "'s", 'ever', 'been', 'seen', 'and', 'we', "'re", 'not', 'going', 'to', 'show', 'you', 'any', 'of', 'it', 'the', 'truth', 'of', 'the', 'matter', 'is', 'that', 'the', 'titanic', 'even', 'though', 'it', "'s", 'breaking', 'all', 'sorts', 'of', 'box', 'office', 'records', 'it', "'s", 'not', 'the', 'most', 'exciting', 'story', 'from', 'the', 'sea', 'and', 'the', 'problem', 'i', 'think', 'is', 'that', 'we', 'take', 'the', 'ocean', 'for', 'granted', 'when', 'you', 'think', 'about', 'it', 'the', 'oceans', 'are', '75', 'percent', 'of', 'the', 'planet', 'most', 'of', 'the', 'planet', 'is', 'ocean', 'water', 'the', 'average', 'depth', 'is', 'about', 'two', 'miles', 'part', 'of', 'the', 'problem', 'i', 'think', 'i

In [ ]:
vocab = {'a': 1, 'b': 2}

vocab['a'] + 1

2

In [ ]:
vocab = {}

for token in tokens:
    if token not in vocab.keys():
        vocab[token] = 1
    else: vocab[token] = vocab[token] + 1

In [ ]:
# print(vocab)
[key for key in vocab.keys() if vocab[key] == 1]

['75',
 'shimmering',
 'surf',
 'volcanoes',
 'biodensity',
 'captivate',
 'sony',
 'proust',
 'lures',
 'dangling',
 'colonial',
 'schooling',
 'bioluminescence',
 'prey',
 'archival',
 'jellies',
 'sizes',
 'gravity',
 '140',
 'octopod',
 'flapping',
 'peek',
 'rooster',
 'formal',
 'caribbean',
 'sonar',
 'jason',
 'headset',
 'woods',
 'sulfide',
 "'70s",
 'volcanism',
 'magma',
 'thrives',
 'eruption',
 'biogenesis',
 'colonies',
 'chemosynthesis',
 'digestive',
 'gill',
 'chemosynthetic',
 'fluid',
 'sugars',
 'shells',
 'fingernails',
 '198',
 'disintegrate',
 'octopods',
 'crush',
 'pepsi',
 'harassing',
 'whack',
 'foodstuff',
 'filaments',
 'clinging',
 'scraping',
 'inhabited',
 'emitting',
 'ir',
 'pagoda',
 'neatest',
 'sulfuric',
 'simplest',
 'lifeblood',
 'wipes',
 'biodensities',
 'nino',
 'tomorrow',
 'preservation',
 'futile',
 '62',
 'optimistic',
 'handicapping',
 'squashing',
 'inadvertently',
 'institutional',
 'scolded',
 'kal',
 'lokvi',
 'babas',
 'dyados',
 '

In [ ]:
import torch


a = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
a.chunk(2, dim = -1)

(tensor([1, 2, 3, 4, 5]), tensor([ 6,  7,  8,  9, 10]))